# Análise dos Sintomas e Evolução dos Casos
* Quais são os sintomas mais comuns entre os casos notificados de dengue (FEBRE, MIALGIA, CEFALEIA, etc.)?
* Existe uma relação entre a presença de sintomas graves (como LEUCOPENIA ou HEMATOLOG) e a evolução dos casos e/ou óbito?
*	Qual é a prevalência de comorbidades (DIABETES, RENAL, HIPERTENSA, etc.) entre os pacientes diagnosticados com dengue? Qual a relação entre a presença de comorbidades e a evolução/óbito?
* Qual é a taxa de hospitalização (HOSPITALIZ) entre os pacientes diagnosticados com dengue?
* Qual é a taxa de mortalidade (TARGET_OBITO) associada à dengue em 2023?
* Existe alguma tendência temporal entre a data do início dos sintomas (DT_SIN_PRI) e a evolução dos casos (EVOLUCAO) ao longo do tempo?

In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [107]:
df = pd.read_csv("dengue_resumido.csv",sep=';')
df = pd.DataFrame(df)
df.head()

,Unnamed: 0,DT_NOTIFIC,ID_MUNICIP,SG_UF_NOT,DT_SIN_PRI,ANO_NASC,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,...,LEUCOPENIA,LACO,DOR_RETRO,DIABETES,HEMATOLOG,HEPATOPAT,RENAL,HIPERTENSA,ACIDO_PEPT,AUTO_IMUNE
0,0,2023-01-01,120070,12,2023-01-01,2005.0,4017,F,5.0,4.0,...,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,1,2023-01-02,120005,12,2023-01-01,1971.0,4051,F,5.0,9.0,...,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,2,2023-01-02,120005,12,2023-01-01,1995.0,4027,F,9.0,9.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,3,2023-01-02,120030,12,2023-01-01,2002.0,4020,M,6.0,4.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,4,2023-01-02,120005,12,2023-01-02,1990.0,4032,M,6.0,9.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


### Dropando o índice

In [108]:
df.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [109]:
df['CS_GESTANT'].unique()

array([ 5.,  9.,  6.,  1.,  4.,  3.,  2., nan])

In [110]:
df['CS_RACA'].unique()

array([ 4.,  9.,  1.,  5.,  2.,  3., nan])

In [111]:
df['CS_ESCOL_N'].unique()

array([nan,  9.,  6.,  8.,  1.,  3., 10.,  4.,  5.,  7.,  2.,  0.])

In [112]:
df.columns

Index(['DT_NOTIFIC', 'ID_MUNICIP', 'SG_UF_NOT', 'DT_SIN_PRI', 'ANO_NASC',
       'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N',
       'HOSPITALIZ', 'EVOLUCAO', 'DT_OBITO', 'FEBRE', 'MIALGIA', 'CEFALEIA',
       'EXANTEMA', 'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE',
       'ARTRALGIA', 'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO',
       'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA',
       'ACIDO_PEPT', 'AUTO_IMUNE'],
      dtype='object')

### Tratamento dos Sintomas

As colunas referentes a sintomas e cormodidades apresentam os seguintes valores:
- 1 = Sim
- 2 = Não

Para melhor entendimento será realizado replace para adequação

In [113]:
colunas = ['FEBRE', 'MIALGIA', 'CEFALEIA',
       'EXANTEMA', 'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE',
       'ARTRALGIA', 'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO',
       'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA',
       'ACIDO_PEPT', 'AUTO_IMUNE']

In [114]:
for valor in colunas:
  df[valor].replace({1:'Sim', 2:'Não'},inplace=True)

C:\Users\Gamer\AppData\Local\Temp\ipykernel_17820\495744036.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[valor].replace({1:'Sim', 2:'Não'},inplace=True)


In [115]:
for valor in colunas:
  df[valor].fillna('Não Informado', inplace=True)

In [116]:
df['VOMITO'].value_counts()

VOMITO
Não              1097815
Sim               357884
Não Informado      52954
Name: count, dtype: int64

In [117]:
df[['FEBRE', 'MIALGIA', 'CEFALEIA',
       'EXANTEMA', 'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE',
       'ARTRALGIA', 'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO',
       'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA',
       'ACIDO_PEPT', 'AUTO_IMUNE']]

,FEBRE,MIALGIA,CEFALEIA,EXANTEMA,VOMITO,NAUSEA,DOR_COSTAS,CONJUNTVIT,ARTRITE,ARTRALGIA,...,LEUCOPENIA,LACO,DOR_RETRO,DIABETES,HEMATOLOG,HEPATOPAT,RENAL,HIPERTENSA,ACIDO_PEPT,AUTO_IMUNE
0,Sim,Não,Sim,Não,Não,Não,Sim,Não,Não,Não,...,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Não
1,Sim,Sim,Não,Não,Não,Não,Não,Não,Não,Não,...,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Não
2,Sim,Sim,Sim,Não,Não,Não,Não,Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
3,Sim,Sim,Sim,Não,Sim,Sim,Não,Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
4,Sim,Não,Sim,Não,Não,Não,Não,Não,Não,Sim,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508648,Sim,Sim,Sim,Não,Não,Não,Não,Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
1508649,Sim,Sim,Sim,Não,Sim,Não,Não,Não,Sim,Sim,...,Não,Não,Sim,Não,Não,Não,Não,Não,Sim,Não
1508650,Sim,Sim,Sim,Não,Não,Não,Sim,Não,Sim,Não,...,Não,Não,Não,Não,Não,Não,Não,Sim,Não,Não
1508651,Sim,Sim,Sim,Não,Sim,Não,Não,Não,Sim,Sim,...,Não,Não,Não,Não,Não,Não,Não,Sim,Não,Não


### Tratamento de Gestante

In [118]:
df['CS_GESTANT'].replace({1:'1º Trimestre',2:'2º Trimestre',3:'3º Trimestre',4:'Idade gestacional ignorada',
                          5:'Não',6:'Não se aplica',9:'Ignorado'},inplace=True)

C:\Users\Gamer\AppData\Local\Temp\ipykernel_17820\1575735799.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['CS_GESTANT'].replace({1:'1º Trimestre',2:'2º Trimestre',3:'3º Trimestre',4:'Idade gestacional ignorada',


In [119]:
df['CS_GESTANT'].fillna('Ignorado', inplace=True)

In [120]:
df['CS_GESTANT'].isnull().sum()

0

### Tratamento Raça

In [121]:
df['CS_RACA'].replace({1:'Branca',2:'Preta',3:'Amarela',4:'Parda',
                          5:'Indígena',9:'Ignorado'},inplace=True)

C:\Users\Gamer\AppData\Local\Temp\ipykernel_17820\996673323.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['CS_RACA'].replace({1:'Branca',2:'Preta',3:'Amarela',4:'Parda',


In [122]:
df['CS_RACA'].fillna('Ignorado', inplace=True)

In [123]:
df['CS_RACA'].value_counts()

CS_RACA
Branca      660516
Parda       526535
Ignorado    237482
Preta        65251
Amarela      15842
Indígena      3027
Name: count, dtype: int64

In [124]:
df['CS_RACA'].isnull().sum()

0

### Tratamento Escolaridade

In [125]:
df['CS_ESCOL_N'].replace({1:'Fundamental incompleto',2:'Fundamental completo',3:'Fundamental incompleto',
                          4:'Fundamental completo',5:'Ensino Médio incompleto',6:'Ensino Médio completo',
                          7:'Ensino Superior incompleto',8:'Ensino superior completo',9:'Ignorado',10:'Não se aplica', 0: 'Ignorado'},inplace=True)

C:\Users\Gamer\AppData\Local\Temp\ipykernel_17820\2399351889.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['CS_ESCOL_N'].replace({1:'Fundamental incompleto',2:'Fundamental completo',3:'Fundamental incompleto',


In [126]:
df['CS_ESCOL_N'].fillna('Ignorado', inplace=True)

In [127]:
df['CS_ESCOL_N'].value_counts()

CS_ESCOL_N
Ignorado                      763293
Ensino Médio completo         249262
Fundamental incompleto        141594
Não se aplica                 100917
Fundamental completo           90212
Ensino Médio incompleto        77064
Ensino superior completo       65083
Ensino Superior incompleto     21228
Name: count, dtype: int64

In [128]:
df['CS_ESCOL_N'].isnull().sum()

0

### Tratamento de Data

In [129]:
import datetime as dt

In [130]:
df[['DT_SIN_PRI','DT_NOTIFIC']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1508653 entries, 0 to 1508652
Data columns (total 2 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   DT_SIN_PRI  1508653 non-null  object
 1   DT_NOTIFIC  1508653 non-null  object
dtypes: object(2)
memory usage: 23.0+ MB


In [131]:
df['DT_SIN_PRI'].isnull().sum()

0

In [132]:
df['DT_NOTIFIC'].isnull().sum()

0

In [133]:
df['DT_SIN_PRI'] = pd.to_datetime(df['DT_SIN_PRI'],format='%Y-%m-%d')
df['DT_NOTIFIC'] = pd.to_datetime(df['DT_NOTIFIC'],format='%Y-%m-%d')

In [134]:
df.sample(10)

,DT_NOTIFIC,ID_MUNICIP,SG_UF_NOT,DT_SIN_PRI,ANO_NASC,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,...,LEUCOPENIA,LACO,DOR_RETRO,DIABETES,HEMATOLOG,HEPATOPAT,RENAL,HIPERTENSA,ACIDO_PEPT,AUTO_IMUNE
735540,2023-05-30,250900,25,2023-05-28,2023.0,2001,F,Não se aplica,Parda,Não se aplica,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
174913,2023-03-18,317020,31,2023-03-14,1990.0,4033,M,Não se aplica,Parda,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
318115,2023-03-08,430680,43,2023-03-06,1967.0,4055,F,Não,Branca,Ignorado,...,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Não
941094,2023-04-21,420200,42,2023-04-18,2013.0,4009,M,Não se aplica,Branca,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
128649,2023-02-10,313820,31,2023-02-08,2002.0,4020,F,Não,Branca,Ensino Médio completo,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
422273,2023-02-01,354075,35,2023-01-18,1981.0,4041,F,Não,Branca,Fundamental incompleto,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
1360184,2023-08-30,520870,52,2023-08-29,1982.0,4041,M,Não se aplica,Amarela,Ignorado,...,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado
10362,2023-02-08,292520,29,2023-02-03,2005.0,4017,F,Não se aplica,Parda,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
236323,2023-01-21,510704,51,2023-01-21,2000.0,4022,M,Não se aplica,Ignorado,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
207034,2023-03-24,312710,31,2023-03-24,2004.0,4018,F,Não,Parda,Ensino Médio incompleto,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não


In [147]:
df['DT_OBITO'] = pd.to_datetime(df['DT_OBITO'],format='%Y-%m-%d')

In [148]:
df['DT_OBITO'].info()

<class 'pandas.core.series.Series'>
Index: 1508616 entries, 0 to 1508652
Series name: DT_OBITO
Non-Null Count  Dtype         
--------------  -----         
1531 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 23.0 MB


### Tratamento de Idade

In [135]:
def calcular_idade(valor, ano_nasc):
    # Converte o valor para string para facilitar o acesso aos dígitos
    valor_str = str(valor)
    
    # Verifica o primeiro dígito
    if valor_str[0] == '4':
        # Retorna os últimos três dígitos como idade
        idade = int(valor_str[1:])
    else:
        # Retorna 1 para outros casos
        idade = 1
    
    #Se a idade for maior do que 100, vamos calulá-la a partir da coluna ANO_NASC
    if idade >= 100:
        idade = 2023 - ano_nasc

    return idade

In [136]:
# Aplica a função em cada valor da coluna NU_IDADE_N
df['NU_IDADE_N'] = df.apply(
    lambda row: calcular_idade(row['NU_IDADE_N'], row['ANO_NASC']), axis=1
)

In [137]:
df['NU_IDADE_N'].value_counts()

NU_IDADE_N
23.0     30483
22.0     30012
25.0     29573
26.0     29087
21.0     28895
         ...  
119.0        1
107.0        1
116.0        1
118.0        1
115.0        1
Name: count, Length: 123, dtype: int64

Criando uma coluna categórica para as faixas etárias

In [138]:
faixas_etarias = {
    "80+": (80, 200),
    "75 a 79": (75, 79),
    "70 a 74": (70, 74),
    "65 a 69": (65, 69),
    "60 a 64": (60, 64),
    "55 a 59": (55, 59),
    "50 a 54": (50, 54),
    "45 a 49": (45, 49),
    "40 a 44": (40, 44),
    "35 a 39": (35, 39),
    "30 a 34": (30, 34),
    "25 a 29": (25, 29),
    "20 a 24": (20, 24),
    "15 a 19": (15, 19),
    "10 a 14": (10, 14),
    "5 a 9": (5, 9),
    "0 a 4": (0, 4)
}

In [139]:
#Criando uma coluna categórica para cada faixa etária
def definir_faixa_etaria(idade):
    for faixa, (min_idade, max_idade) in faixas_etarias.items():
        if min_idade <= idade <= max_idade:
            return faixa
    return 'Idade não informada'

In [140]:
df.loc[:, 'Faixa Etária'] = df['NU_IDADE_N'].apply(definir_faixa_etaria)
df.head()

,DT_NOTIFIC,ID_MUNICIP,SG_UF_NOT,DT_SIN_PRI,ANO_NASC,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,...,LACO,DOR_RETRO,DIABETES,HEMATOLOG,HEPATOPAT,RENAL,HIPERTENSA,ACIDO_PEPT,AUTO_IMUNE,Faixa Etária
0,2023-01-01,120070,12,2023-01-01,2005.0,17.0,F,Não,Parda,Ignorado,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,15 a 19
1,2023-01-02,120005,12,2023-01-01,1971.0,51.0,F,Não,Ignorado,Ignorado,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,50 a 54
2,2023-01-02,120005,12,2023-01-01,1995.0,27.0,F,Ignorado,Ignorado,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,25 a 29
3,2023-01-02,120030,12,2023-01-01,2002.0,20.0,M,Não se aplica,Parda,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,20 a 24
4,2023-01-02,120005,12,2023-01-02,1990.0,32.0,M,Não se aplica,Ignorado,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,30 a 34


In [ ]:
df_nulo = df[df['NU_IDADE_N'].isnull()]
df_nulo

In [142]:
df.dropna(subset = "NU_IDADE_N", inplace=True)

In [143]:
df['NU_IDADE_N'].isnull().sum()

0

### Tratamento Sexo

In [144]:
df['CS_SEXO'].replace({'M':'Masculino', 'F':'Feminino', 'I':'Ignorado'},inplace=True)

C:\Users\Gamer\AppData\Local\Temp\ipykernel_17820\1236526691.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['CS_SEXO'].replace({'M':'Masculino', 'F':'Feminino', 'I':'Ignorado'},inplace=True)


In [145]:
df['CS_SEXO'].fillna('Ignorado', inplace=True)

In [146]:
df['CS_SEXO'].isnull().sum()

0

### Exportando para CSV

In [149]:
df.head()

,DT_NOTIFIC,ID_MUNICIP,SG_UF_NOT,DT_SIN_PRI,ANO_NASC,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,...,LACO,DOR_RETRO,DIABETES,HEMATOLOG,HEPATOPAT,RENAL,HIPERTENSA,ACIDO_PEPT,AUTO_IMUNE,Faixa Etária
0,2023-01-01,120070,12,2023-01-01,2005.0,17.0,Feminino,Não,Parda,Ignorado,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,15 a 19
1,2023-01-02,120005,12,2023-01-01,1971.0,51.0,Feminino,Não,Ignorado,Ignorado,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,50 a 54
2,2023-01-02,120005,12,2023-01-01,1995.0,27.0,Feminino,Ignorado,Ignorado,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,25 a 29
3,2023-01-02,120030,12,2023-01-01,2002.0,20.0,Masculino,Não se aplica,Parda,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,20 a 24
4,2023-01-02,120005,12,2023-01-02,1990.0,32.0,Masculino,Não se aplica,Ignorado,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,30 a 34


In [150]:
df.to_csv('dengue_resumido_V3.csv',sep=';',index=False)


In [151]:
df2 = pd.read_csv('dengue_resumido_V3.csv',sep=';')
df2.head()

,DT_NOTIFIC,ID_MUNICIP,SG_UF_NOT,DT_SIN_PRI,ANO_NASC,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,...,LACO,DOR_RETRO,DIABETES,HEMATOLOG,HEPATOPAT,RENAL,HIPERTENSA,ACIDO_PEPT,AUTO_IMUNE,Faixa Etária
0,2023-01-01,120070,12,2023-01-01,2005.0,17.0,Feminino,Não,Parda,Ignorado,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,15 a 19
1,2023-01-02,120005,12,2023-01-01,1971.0,51.0,Feminino,Não,Ignorado,Ignorado,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,50 a 54
2,2023-01-02,120005,12,2023-01-01,1995.0,27.0,Feminino,Ignorado,Ignorado,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,25 a 29
3,2023-01-02,120030,12,2023-01-01,2002.0,20.0,Masculino,Não se aplica,Parda,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,20 a 24
4,2023-01-02,120005,12,2023-01-02,1990.0,32.0,Masculino,Não se aplica,Ignorado,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,30 a 34
